In [1]:
import matplotlib
matplotlib.use('Agg')  # plotting backend compatible with screen
import sys
import scanpy as sc

import six
import numba
import collections
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.sparse as sp_sparse
import tables
import pickle

sc.settings.verbosity = 2  # show logging output
sc.settings.autosave = True  # save figures, do not show them
sc.settings.set_figure_params(dpi=400)  # set sufficiently high resolution for saving

# Load data

In [ ]:
! wget --no-check-certificate https://s3-us-west-2.amazonaws.com/10x.files/samples/cell/1M_neurons/1M_neurons_filtered_gene_bc_matrices_h5.h5

In [27]:
adata_full = sc.read_10x_h5('1M_neurons_filtered_gene_bc_matrices_h5.h5')

reading 1M_neurons_filtered_gene_bc_matrices_h5.h5


Variable names are not unique. To make them unique, call `.var_names_make_unique`.


 (0:01:29)


# Scanpy pipeline runtime on CPU ~33min 

In [4]:
# function to run recipes, generate plots, using the CPU (not leveraging the GPU)

def basic_analysis_cpugraph(anndata_object):
    sc.pp.subsample(anndata_object, fraction=0.5)
    sc.pp.recipe_zheng17(anndata_object)
    sc.pp.neighbors(anndata_object)
    sc.tl.louvain(anndata_object)
    sc.tl.umap(anndata_object)
    sc.tl.rank_genes_groups(anndata_object, 'louvain')
    sc.pl.rank_genes_groups(anndata_object, save='_130K.pdf')

In [5]:
%%time 

basic_analysis_cpugraph(adata_full)

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


running recipe zheng17
filtered out 4536 genes that are detected in less than 1 counts


Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


normalizing counts per cell
    finished (0:00:05)
extracting highly variable genes
    finished (0:00:05)
normalizing counts per cell
    finished (0:00:00)
... as `zero_center=True`, sparse input is densified and may lead to large memory consumption
    finished (0:00:36)
computing neighbors
         Falling back to preprocessing with `sc.pp.pca` and default params.
computing PCA
    with n_comps=50
    finished (0:00:14)
    finished (0:09:35)
running Louvain clustering
    using the "louvain" package of Traag (2017)
    finished (0:05:06)
computing UMAP
    finished (0:15:48)
ranking genes


/home/jovyan/.local/lib/python3.7/site-packages/scanpy/tools/_rank_genes_groups.py:252: RuntimeWarning: invalid value encountered in log2
  rankings_gene_logfoldchanges.append(np.log2(foldchanges[global_indices]))


    finished (0:01:33)
CPU times: user 43min 36s, sys: 1min 14s, total: 44min 51s
Wall time: 33min 11s


# Scanpy pipeline runtime on GPU ~7min 

In [ ]:
# read in data again, scanpy pipeline alters the anndata object 

adata_full = sc.read_10x_h5('1M_neurons_filtered_gene_bc_matrices_h5.h5')

In [9]:
# function to run recipes, generate plots, using our GPU

def basic_analysis_gpugraph(anndata_object):
    sc.pp.subsample(anndata_object, fraction=0.5)
    sc.pp.recipe_zheng17(anndata_object)
    sc.pp.neighbors(anndata_object, method='rapids')
    sc.tl.louvain(anndata_object, flavor='rapids')
    sc.tl.umap(anndata_object, method='rapids')
    sc.tl.rank_genes_groups(anndata_object, 'louvain') 
    sc.pl.umap(anndata_object, color='louvain', save='_130K.png')
    sc.pl.rank_genes_groups(anndata_object, save='_130K.pdf')

In [11]:
%%time 

basic_analysis_gpugraph(adata_full)

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


running recipe zheng17
filtered out 4536 genes that are detected in less than 1 counts


Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


normalizing counts per cell
    finished (0:00:05)
extracting highly variable genes
    finished (0:00:05)
normalizing counts per cell
    finished (0:00:00)
... as `zero_center=True`, sparse input is densified and may lead to large memory consumption
    finished (0:00:36)
computing neighbors
         Falling back to preprocessing with `sc.pp.pca` and default params.
computing PCA
    with n_comps=50
    finished (0:00:13)
    finished (0:01:06)
running Louvain clustering
    using the "louvain" package of rapids
    finished (0:00:01)
computing UMAP


/home/jovyan/.local/lib/python3.7/site-packages/scanpy/tools/_umap.py:197: UserWarning: Parameter should_downcast is deprecated, use convert_dtype in fit, fit_transform and transform  methods instead. 
  verbose=settings.verbosity > 3,
/home/jovyan/.local/lib/python3.7/site-packages/scanpy/tools/_umap.py:199: UserWarning: Parameter should_downcast is deprecated, use convert_dtype in fit, fit_transform and transform  methods instead. 
  X_umap = umap.fit_transform(X_contiguous)


    finished (0:02:15)
ranking genes


/home/jovyan/.local/lib/python3.7/site-packages/scanpy/tools/_rank_genes_groups.py:252: RuntimeWarning: invalid value encountered in log2
  rankings_gene_logfoldchanges.append(np.log2(foldchanges[global_indices]))


    finished (0:02:29)
CPU times: user 6min 11s, sys: 1min 54s, total: 8min 5s
Wall time: 7min 3s


# running the same scanpy pipeline takes ~30min on CPU vs. ~7min on GPU